In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

In [ ]:
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
# Load dataset.
dftrain = pd.read_csv('train.csv') #train model
dfeval = pd.read_csv('eval.csv') #test data
# dftrain.head()
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')


In [ ]:
dftrain.describe()

In [ ]:
dftrain.shape

In [ ]:
dftrain.age.hist(bins=20)

In [ ]:
dfeval.shape

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [ ]:
print(feature_columns)

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [ ]:
ds = make_input_fn(dftrain, y_train, batch_size=10)()
count = 0
for feature_batch, label_batch in ds.take(1):
  count += 1
  # print('Some feature keys:', feature_batch)
  # print()
  # print('A batch of class:', feature_batch['class'].numpy())
  # print()
  # print('A batch of Labels:', label_batch.numpy())
print(count, len(ds))

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

In [ ]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[4])
print(y_eval[4])
print(result[4]['probabilities'][1])